In [2]:
import requests
import time
from datetime import datetime
def get_market_by_slug(slug: str) -> dict:
    """Fetch market details directly by slug (no searching)"""
    response = requests.get(
        f"https://gamma-api.polymarket.com/markets",
        params={"slug": slug}
    )
    markets = response.json()
    return markets[0] if markets else None
def get_btc_15m_market(timestamp: int) -> dict:
    """Get BTC 15-min market for a specific timestamp"""
    slug = f"btc-updown-15m-{timestamp}"
    return get_market_by_slug(slug)
def get_next_market_timestamp(current_ts: int) -> int:
    """Calculate next 15-min market timestamp"""
    return current_ts + 900
# Example: Get current market
current_ts = 1766343600
market = get_btc_15m_market(current_ts)
if market:
    print(f"Question: {market['question']}")
    print(f"End Date: {market['endDate']}")
    print(f"Condition ID: {market['conditionId']}")
    print(f"Token IDs: {market['clobTokenIds']}")  # This is what WebSocket needs

    yes_token_id = market['clobTokenIds'][0]
    no_token_id = market['clobTokenIds'][1]

Question: Bitcoin Up or Down - December 21, 2:00PM-2:15PM ET
End Date: 2025-12-21T19:15:00Z
Condition ID: 0x9398735f60e0a2cb7f2d1da3dd81655d30ad97ffce7fe0df47732681d7212c94
Token IDs: ["3223633815316585577932633765008866706669054603038585136417456806418942387729", "23921371670884067749883484489602397785066176993399768989861335947727034398965"]


In [6]:
import json
import threading
import requests
import pandas as pd
from websocket import WebSocketApp
from datetime import datetime
import time

# ============ CONFIGURATION (UPDATE THESE) ============
TOKEN_ID = "3223633815316585577932633765008866706669054603038585136417456806418942387729"                           # From Gamma API
MARKET_SLUG = "btc-updown-15m-1766343600"                 # Current market slug
MARKET_END_UNIX = 1766343600 + 900                        # Slug timestamp + 900 seconds
USER_ADDRESS = "0x6031b6eed1c97e853c6e0f03ad3ce3529351f96d"

# ============ SHARED STATE ============
bids = []
asks = []
btc_price = None

# ============ DATA STORAGE ============
orderbook_snapshots = []
user_trades = []
seen_hashes = set()

In [8]:
# ============ WEBSOCKET: Live Orderbook ============
def on_open(ws):
    print("[WS] Connected")
    ws.send(json.dumps({"assets_ids": [TOKEN_ID], "type": "market"}))

def on_message(ws, message):
    global bids, asks
    if message == "PONG":
        return
    data = json.loads(message)
    events = data if isinstance(data, list) else [data]
    for event in events:
        if event.get("event_type") == "book" and event.get("asset_id") == TOKEN_ID:
            bids = event.get("bids", [])
            asks = event.get("asks", [])

ws = WebSocketApp(
    "wss://ws-subscriptions-clob.polymarket.com/ws/market",
    on_open=on_open,
    on_message=on_message
)
threading.Thread(target=ws.run_forever, daemon=True).start()
print("[WS] Orderbook thread started")

[WS] Orderbook thread started


In [24]:
# ============ BTC PRICE POLLER (CoinGecko) ============
def btc_poller():
    global btc_price
    while True:
        try:
            r = requests.get(
                "https://api.coingecko.com/api/v3/simple/price",
                params={"ids": "bitcoin", "vs_currencies": "usd"},
                timeout=5
            )
            btc_price = float(r.json()["bitcoin"]["usd"])
        except Exception as e:
            print(f"[BTC] Error: {e}")
        time.sleep(1)  # CoinGecko rate limit is stricter

threading.Thread(target=btc_poller, daemon=True).start()
print("[BTC] Price poller started")

[BTC] Price poller started


In [12]:
# ============ ORDERBOOK SAMPLER (10x per second) ============
def sampler():
    while True:
        now_unix = int(time.time())
        
        if bids and asks and btc_price:
            # Top 5 depth
            bid_depth_5 = sum(float(b["size"]) for b in bids[-5:])
            ask_depth_5 = sum(float(a["size"]) for a in asks[-5:])
            
            orderbook_snapshots.append({
                "timestamp_unix": now_unix,
                "market_slug": MARKET_SLUG,
                "best_bid": float(bids[-1]["price"]),
                "best_ask": float(asks[-1]["price"]),
                "spread": float(asks[-1]["price"]) - float(bids[-1]["price"]),
                "bid_size": float(bids[-1]["size"]),
                "ask_size": float(asks[-1]["size"]),
                "bid_depth_5": bid_depth_5,
                "ask_depth_5": ask_depth_5,
                "seconds_to_expiry": max(0, MARKET_END_UNIX - now_unix),
                "btc_price": btc_price
            })
        time.sleep(0.1)

threading.Thread(target=sampler, daemon=True).start()
print("[SAMPLER] Orderbook sampler started")

[SAMPLER] Orderbook sampler started


In [14]:
# ============ TRADE POLLER ============
def trade_poller():
    while True:
        try:
            r = requests.get(
                "https://data-api.polymarket.com/trades",
                params={"user": USER_ADDRESS, "limit": 100},
                timeout=2
            )
            for trade in r.json():
                tx_hash = trade.get("transactionHash")
                slug = trade.get("slug", "")
                
                if "btc-updown-15m" not in slug:
                    continue
                    
                if tx_hash and tx_hash not in seen_hashes:
                    seen_hashes.add(tx_hash)
                    user_trades.append({
                        "timestamp_unix": trade.get("timestamp"),
                        "market_slug": slug,
                        "side": trade.get("side"),
                        "price": trade.get("price"),
                        "size": trade.get("size"),
                        "outcome": trade.get("outcome"),
                        "tx_hash": tx_hash
                    })
                    print(f"[TRADE] {trade.get('side')} {trade.get('size')} @ {trade.get('price')}")
        except Exception as e:
            print(f"[TRADE] Error: {e}")
        time.sleep(0.1)

threading.Thread(target=trade_poller, daemon=True).start()
print("[TRADE] Trade poller started")

[TRADE] Trade poller started


In [16]:
# View orderbook snapshots
ob_df = pd.DataFrame(orderbook_snapshots)
print(f"Orderbook snapshots: {len(ob_df)}")
display(ob_df.tail(10))

Orderbook snapshots: 0


""


In [18]:
# View orderbook snapshots
ob_df = pd.DataFrame(orderbook_snapshots)
print(f"Orderbook snapshots: {len(ob_df)}")
display(ob_df.tail(5))

Orderbook snapshots: 0


""


In [20]:
# View user trades
trades_df = pd.DataFrame(user_trades)
print(f"User trades: {len(trades_df)}")
display(trades_df.tail(10))

User trades: 56


,timestamp_unix,market_slug,side,price,size,outcome,tx_hash
46,1766343826,btc-updown-15m-1766343600,BUY,0.71,20.00,Down,0xc5165eeab5e219533f5c57a6650044a595e766b47c56...
47,1766343812,btc-updown-15m-1766343600,BUY,0.41,20.00,Up,0xd1998f14c8154d553a3a59ff4c3a3a5ac884124b5df9...
48,1766343808,btc-updown-15m-1766343600,BUY,0.55,20.00,Down,0x17d99604c7b6d0eb2e8d88b245eb17c44bf1782d9d50...
49,1766343808,btc-updown-15m-1766343600,BUY,0.54,20.00,Down,0x9fc8d0dc7cb9e6aefd82f16c24de020cd53b6bb50433...
50,1766343808,btc-updown-15m-1766343600,BUY,0.54,5.00,Down,0x87fc50f890090c5cdaa0bfbf86ff5953cb51215bf72a...
51,1766343808,btc-updown-15m-1766343600,BUY,0.55,20.00,Down,0x64d7161dcbad5776bfd42da4afc6b01faaa56c44c5b9...
52,1766343808,btc-updown-15m-1766343600,BUY,0.54,20.00,Down,0x7dd3eee586224cc1f561de7439addbe67b2169200ed3...
53,1766343808,btc-updown-15m-1766343600,BUY,0.54,6.81,Down,0x24b3add1dcaff5171507460887203b0ef04f81435e9b...
54,1766343808,btc-updown-15m-1766343600,BUY,0.55,20.00,Down,0xb385dbac567c409d3234b84c9aff735aba61d5eca458...
55,1766343800,btc-updown-15m-1766343600,BUY,0.50,14.80,Up,0xc139ff887d3b70c3e903f0cd5488aa3d5d2bc70ccf23...


In [22]:
print(f"bids: {len(bids)} levels")
print(f"asks: {len(asks)} levels")
print(f"btc_price: {btc_price}")
print(f"snapshots: {len(orderbook_snapshots)}")

bids: 30 levels
asks: 69 levels
btc_price: None
snapshots: 0


In [26]:
print(f"btc_price: {btc_price}")

btc_price: 88375.0


In [28]:
# View orderbook snapshots
ob_df = pd.DataFrame(orderbook_snapshots)
print(f"Orderbook snapshots: {len(ob_df)}")
display(ob_df.tail(5))

Orderbook snapshots: 176


,timestamp_unix,market_slug,best_bid,best_ask,spread,bid_size,ask_size,bid_depth_5,ask_depth_5,seconds_to_expiry,btc_price
171,1766344053,btc-updown-15m-1766343600,0.16,0.18,0.02,192.92,118.37,2152.63,2016.33,447,88375.0
172,1766344053,btc-updown-15m-1766343600,0.16,0.18,0.02,192.92,118.37,2152.63,2016.33,447,88375.0
173,1766344053,btc-updown-15m-1766343600,0.16,0.18,0.02,192.92,118.37,2152.63,2016.33,447,88375.0
174,1766344053,btc-updown-15m-1766343600,0.16,0.18,0.02,39.31,118.37,1986.72,1830.33,447,88375.0
175,1766344053,btc-updown-15m-1766343600,0.16,0.18,0.02,4.23,118.37,1814.72,1698.41,447,88375.0


In [30]:
# View orderbook snapshots
ob_df = pd.DataFrame(orderbook_snapshots)
print(f"Orderbook snapshots: {len(ob_df)}")
display(ob_df.tail(5))

Orderbook snapshots: 215


,timestamp_unix,market_slug,best_bid,best_ask,spread,bid_size,ask_size,bid_depth_5,ask_depth_5,seconds_to_expiry,btc_price
210,1766344057,btc-updown-15m-1766343600,0.17,0.19,0.02,264.64,10.00,1918.10,949.04,443,88375.0
211,1766344057,btc-updown-15m-1766343600,0.17,0.19,0.02,264.64,10.00,1918.10,949.04,443,88375.0
212,1766344057,btc-updown-15m-1766343600,0.17,0.19,0.02,264.64,10.00,1918.10,949.04,443,88375.0
213,1766344057,btc-updown-15m-1766343600,0.17,0.19,0.02,246.35,185.92,2018.81,1155.88,443,88375.0
214,1766344057,btc-updown-15m-1766343600,0.17,0.19,0.02,246.35,185.92,2018.81,1155.88,443,88375.0


In [32]:
# View user trades
trades_df = pd.DataFrame(user_trades)
print(f"User trades: {len(trades_df)}")
display(trades_df.tail(10))

User trades: 66


,timestamp_unix,market_slug,side,price,size,outcome,tx_hash
56,1766343968,btc-updown-15m-1766343600,BUY,0.61,20.00,Down,0x687b5e7e51d6458f72d329b678373a6bce0603d9716a...
57,1766343948,btc-updown-15m-1766343600,BUY,0.41,20.00,Up,0x0f5b941197d8edb1ffae716a079bc5c3113ad59834bf...
58,1766343938,btc-updown-15m-1766343600,BUY,0.53,20.00,Down,0xbd71f3b14e482faed5ff6f43453d265364596e3cae22...
59,1766343920,btc-updown-15m-1766343600,BUY,0.40,20.00,Up,0xf84f0e7e56f32f7e769eb0a6506d884e7dfee96b0903...
60,1766343920,btc-updown-15m-1766343600,BUY,0.40,11.25,Up,0x341cf2893abefa00bd430c86c2a40c407bb4b1428530...
61,1766343920,btc-updown-15m-1766343600,BUY,0.40,20.00,Up,0x9c72ef017a76f4711e0d24fe73d2fcff1aab0e15be67...
62,1766343920,btc-updown-15m-1766343600,BUY,0.40,20.00,Up,0x68317824e237a83535c0f588785612ef20e55683a3d3...
63,1766343912,btc-updown-15m-1766343600,BUY,0.38,20.00,Up,0xce1a9a6f25e5e67503fb2e31d222eff56601b24a3613...
64,1766343912,btc-updown-15m-1766343600,BUY,0.38,20.00,Up,0x77e627de38f6d72022b51aa3629b41ca0b3ed57dc0b0...
65,1766343886,btc-updown-15m-1766343600,BUY,0.21,5.00,Up,0x1c33cf400c364f58813c951cf775cc91b77f74cc3ac5...


In [34]:
# Status check
print(f"BTC Price: ${btc_price:,.2f}" if btc_price else "BTC: waiting...")
print(f"Book levels: {len(bids)} bids, {len(asks)} asks")
print(f"Snapshots: {len(orderbook_snapshots)}")
print(f"Trades: {len(user_trades)}")

BTC Price: $88,375.00
Book levels: 15 bids, 84 asks
Snapshots: 401
Trades: 83


In [36]:
trades_df.head()

,timestamp_unix,market_slug,side,price,size,outcome,tx_hash
0,1766343790,btc-updown-15m-1766343600,BUY,0.49,17.85,Down,0x380c367140189c596c4aab48e5dfcfb3e76db46bc7e8...
1,1766343760,btc-updown-15m-1766343600,BUY,0.51,7.00,Down,0xfd9eb535415bbc6af37ec48aa1c07924bda7378f9486...
2,1766343746,btc-updown-15m-1766343600,BUY,0.55,20.00,Up,0x3b37942beea1258efe36a2ae96e60978c2903e04c031...
3,1766343746,btc-updown-15m-1766343600,BUY,0.55,20.00,Up,0x84c4f60e6b3db51cd9f890e6bad85da2f84ccda35683...
4,1766343732,btc-updown-15m-1766343600,BUY,0.42,5.00,Down,0xd5e1bca14356826fb85e3c55a63e857de4acb81d99ee...


In [38]:
import pandas as pd

ob_df = pd.DataFrame(orderbook_snapshots)
trades_df = pd.DataFrame(user_trades)

merged = pd.merge_asof(
    trades_df.sort_values("timestamp_unix"),
    ob_df.sort_values("timestamp_unix"),
    on="timestamp_unix",
    direction="backward",
    suffixes=("_trade", "_book")
)

print(f"Merged rows: {len(merged)}")
display(merged.head(20))

Merged rows: 105


,timestamp_unix,market_slug_trade,side,price,size,outcome,tx_hash,market_slug_book,best_bid,best_ask,spread,bid_size,ask_size,bid_depth_5,ask_depth_5,seconds_to_expiry,btc_price
0,1766343312,btc-updown-15m-1766342700,BUY,0.980000,18.00,Up,0x930c9f96de506f82fc2b444c4bef337026c746670978...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1766343312,btc-updown-15m-1766342700,BUY,0.980000,18.00,Up,0x13ba849cca2dfad02e4f2e8ee671c98fe4af702b6c10...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1766343312,btc-updown-15m-1766342700,BUY,0.980000,18.00,Up,0xfba712938d46f8e5355266277680a79dac21c7c0e448...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1766343312,btc-updown-15m-1766342700,BUY,0.980000,18.00,Up,0x30eecec568b5918b5e8a572ffc459b8a15a18ae84443...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766343622,btc-updown-15m-1766343600,BUY,0.384172,8.58,Down,0x0bb26decd377214e3e029a02fc4e096519ff4bc4bd48...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1766343624,btc-updown-15m-1766343600,BUY,0.420000,11.00,Down,0x83d40bea060e7b303f959007d9ce17cfba26e8fa9a1a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1766343628,btc-updown-15m-1766343600,BUY,0.400000,14.00,Down,0x689b95b6593d7732f09e15be6f61a409ae23de03cac0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1766343638,btc-updown-15m-1766343600,BUY,0.630000,8.87,Up,0xb8bdbdd6eaee4c3e7ab5e777cf0f553ef2574d180ef7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1766343644,btc-updown-15m-1766343600,BUY,0.370000,2.69,Down,0xa825709ceb7a271548a05ae2625d03e8045555ee1025...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1766343656,btc-updown-15m-1766343600,BUY,0.430000,20.00,Down,0xac074397d1d2c79903719b35badd3bc14aa654959ee0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
